In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_data, upload_model
import catboost as cb
from sklearn.metrics import precision_score
import joblib

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get feature/target data from hopsworks, so we can train a model

In [3]:
eth_ohlc = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)
eth_ohlc = eth_ohlc[:-1] # the last row contains NA values for tomorrow's price and target, so we have to ommit it here
eth_ohlc

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.13s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,1.0,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.0,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,0.0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,2024-05-08,3013.0000,3036.0000,2944.0000,2975.0000,1.020000e+10,3400618.0,3.675000e+11,1.0,0.023697,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3079,2024-05-09,2977.0000,3055.0000,2958.0000,3036.0000,9.600000e+09,3190231.0,3.670000e+11,0.0,-0.018445,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3080,2024-05-10,3035.0000,3050.0000,2889.0000,2910.0000,1.060000e+10,3557434.0,3.650000e+11,0.0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.0000,2935.0000,2894.0000,2911.0000,5.900000e+09,2033385.0,3.569000e+11,1.0,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


### Define the model and obtain a backtested precision score (precision score = out of all the times we predicted the price would go up, what proportion of the time did it actually go up the next day)

In [7]:
model = cb.CatBoostClassifier(iterations=300, learning_rate=.019, depth=6, verbose=False, random_seed=1)

predictors_96 = ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_80th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 'spread_rsi_5', 'last_5_day_60th_pct_price_change_count', 'last_5_day_70th_pct_price_change_count', 'rsi_10', 'volume_rsi_10', 'spread_rsi_10', 'last_10_day_40th_to_50th_pct_price_change_count', 'last_10_day_60th_pct_price_change_count', 'last_10_day_70th_to_80th_pct_volume_change_count', 'last_10_day_80th_to_90th_pct_spread_count', 'last_10_day_40th_pct_spread_count', 'last_10_day_50th_pct_spread_count', 'last_10_day_70th_pct_spread_count', 'last_10_day_80th_pct_spread_count', 'rsi_25', 'volume_rsi_25', 'spread_rsi_25', 'last_25_day_40th_to_50th_pct_price_change_count', 'last_25_day_70th_to_80th_pct_price_change_count', 'last_25_day_80th_to_90th_pct_price_change_count', 'last_25_day_40th_pct_price_change_count', 'last_25_day_50th_pct_price_change_count', 'last_25_day_80th_pct_price_change_count', 'last_25_day_40th_to_50th_pct_volume_change_count', 'last_25_day_50th_to_60th_pct_volume_change_count', 'last_25_day_60th_to_70th_pct_volume_change_count', 'last_25_day_70th_to_80th_pct_volume_change_count', 'last_25_day_80th_to_90th_pct_volume_change_count', 'last_25_day_50th_pct_volume_change_count', 'last_25_day_60th_pct_volume_change_count', 'last_25_day_70th_pct_volume_change_count', 'last_25_day_80th_pct_volume_change_count', 'last_25_day_40th_to_50th_pct_spread_count', 'last_25_day_50th_to_60th_pct_spread_count', 'last_25_day_60th_to_70th_pct_spread_count', 'last_25_day_70th_to_80th_pct_spread_count', 'last_25_day_80th_to_90th_pct_spread_count', 'last_25_day_40th_pct_spread_count', 'last_25_day_60th_pct_spread_count', 'last_25_day_70th_pct_spread_count', 'last_25_day_90th_pct_spread_count', 'rsi_50', 'last_50_day_50th_to_60th_pct_price_change_count', 'last_50_day_60th_to_70th_pct_price_change_count', 'last_50_day_70th_to_80th_pct_price_change_count', 'last_50_day_40th_pct_price_change_count', 'last_50_day_50th_pct_price_change_count', 'last_50_day_70th_pct_price_change_count', 'last_50_day_40th_to_50th_pct_volume_change_count', 'last_50_day_50th_to_60th_pct_volume_change_count', 'last_50_day_50th_pct_volume_change_count', 'last_50_day_80th_pct_volume_change_count', 'last_50_day_40th_to_50th_pct_spread_count', 'last_50_day_50th_to_60th_pct_spread_count', 'last_50_day_60th_to_70th_pct_spread_count', 'last_50_day_70th_to_80th_pct_spread_count', 'last_50_day_80th_to_90th_pct_spread_count', 'last_50_day_40th_pct_spread_count', 'last_50_day_50th_pct_spread_count', 'last_50_day_70th_pct_spread_count', 'last_50_day_90th_pct_spread_count', 'rsi_100', 'last_100_day_60th_to_70th_pct_price_change_count', 'last_100_day_80th_to_90th_pct_price_change_count', 'last_100_day_90th_to_95th_pct_price_change_count', 'last_100_day_40th_pct_price_change_count', 'last_100_day_50th_pct_price_change_count', 'last_100_day_60th_pct_price_change_count', 'last_100_day_80th_pct_price_change_count', 'last_100_day_40th_to_50th_pct_volume_change_count', 'last_100_day_50th_to_60th_pct_volume_change_count', 'last_100_day_60th_to_70th_pct_volume_change_count', 'last_100_day_70th_to_80th_pct_volume_change_count', 'last_100_day_80th_to_90th_pct_volume_change_count', 'last_100_day_40th_pct_volume_change_count', 'last_100_day_60th_pct_volume_change_count', 'last_100_day_80th_pct_volume_change_count', 'last_100_day_90th_pct_volume_change_count', 'last_100_day_40th_to_50th_pct_spread_count', 'last_100_day_50th_to_60th_pct_spread_count', 'last_100_day_60th_to_70th_pct_spread_count', 'last_100_day_70th_to_80th_pct_spread_count', 'last_100_day_90th_to_95th_pct_spread_count', 'last_100_day_40th_pct_spread_count', 'last_100_day_50th_pct_spread_count', 'last_100_day_60th_pct_spread_count', 'last_100_day_80th_pct_spread_count', 'last_100_day_90th_pct_spread_count']
response_var = 'tmw_1_0_percent_increase_binary'

threshold = 0.55

In [11]:
def get_precision_score(data, model, predictors, response_var, threshold, start=1000, step=100):
    
    def predict(train, test, predictors, model):
        model.fit(train[predictors], train[response_var])
        probability = model.predict_proba(test[predictors])[:,1]
        proba_series = pd.Series(probability, index=test.index, name="probability")
        combined = pd.concat([test[response_var], test['tmw_percent_increase_to_avg_high_low'], proba_series], axis=1)
        return combined

    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+100)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

    data_with_predictions = pd.concat(all_predictions)

    col_name = f'pred_{threshold}'
    data_with_predictions[col_name] = data_with_predictions['probability'] >= threshold

    ppv = precision_score(data_with_predictions[response_var], data_with_predictions[col_name])
    num_1s = data_with_predictions[col_name].value_counts().get(1, 0)
    
    return ppv, num_1s

In [13]:
ppv, num_1_preds = get_precision_score(eth_ohlc, model, predictors_96, response_var, threshold)
ppv, num_1_preds

(0.6163069544364509, 417)

### Fit the model on all the data, then save it into a file as a serialized model
##### joblib is for serializing models in order to save them to disk. Note that pickle and joblib are both modules that serialize Python objects into a byte stream, and deserializes the byte stream back to Python objects. However, joblib is useful when the model involves a lot of data and arrays, and is generally recommended for scikit-learn models because of its efficiency with these objects.

In [16]:
model.fit(eth_ohlc[predictors_96], eth_ohlc[response_var])

MODEL_NAME = 'catboost_eth_returns'

joblib.dump(model, f'../models/ohlc/{MODEL_NAME}.pkl')

['../models/ohlc/catboost_eth_returns.pkl']

### Upload the serialized model into hopsworks

In [17]:
last_date = pd.to_datetime(eth_ohlc['date'].iloc[-1]).tz_localize(None)
trained_till = last_date.strftime('%m/%d/%Y')
trained_till

'05/12/2024'

In [18]:
# def upload_model(model_name, model_description, metric_type, metric_value):
upload_model(MODEL_NAME, f"iterations 300, learning_rate=0.019, depth=6, Predict {response_var}, Trained Until {trained_till}, PPV Threshold Used {threshold}", "PPV", ppv)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 352063/352063 elapsed<00:01 remaining<00:00  4.73it/s]
Model export complete: 100%|██████████| 6/6 [00:07<00:00,  1.24s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/711829/models/catboost_eth_returns/1
